In [9]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.general as gen
import modules.gait_metrics as gm
import modules.assign_sides as asi

In [10]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [11]:
file_name = '2014-12-08_P004_Post_004'

In [16]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_head_feet = pd.read_pickle(best_pos_paths[0])

In [17]:
df_hypo.L_FOOT[200]


array([[-67.3317, -56.3553, 244.679],
       [-45.9189, -65.4187, 263.42900000000003],
       [-49.981, -65.5531, 262.126],
       [-66.737, -58.9848, 246.347],
       [-47.8853, -65.3711, 262.991],
       [-64.3456, -62.31100000000001, 249.727],
       [-65.7141, -60.9874, 248.03900000000002]], dtype=object)

## Split trial into separate passes

In [18]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_head_feet.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = gen.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = gen.group_by_label(df_head_feet, labels)

In [19]:
df_pass = list(pass_dfs)[0]

In [20]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Foot signal

In [21]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

In [22]:
plt.plot(signal_l)

## Assign DataFrame for plotting

In [23]:
df = df_assigned
dim_order = [0, 2, 1]


## View one frame

In [104]:
frame = 250

fig, ax = plt.subplots()

ax = Axes3D(fig)

positions = df.loc[frame]

points = np.stack(positions)
points_to_plot = points[:, dim_order]

head, foot_l, foot_r = points_to_plot

pl.scatter3(ax, head, color='k')
pl.scatter3(ax, foot_l, color='b')
pl.scatter3(ax, foot_r, color='r')    

# Plot hypotheses
df_hypo_frame = df_hypo.loc[frame]
pl.scatter3(ax, df_hypo_frame.L_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)
pl.scatter3(ax, df_hypo_frame.R_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)

ax.set_xlim3d(-200, 200)
ax.set_ylim3d(100, 300)
ax.set_zlim3d(-100, 100)

plt.show()

## View multiple frames

In [105]:
fig, ax = plt.subplots()

ax = Axes3D(fig)

frames = df.index.values

for frame in frames:
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue
    
    points = np.stack(positions)
    points_to_plot = points[:, dim_order]

    head, foot_l, foot_r = points_to_plot

    pl.scatter3(ax, head, color='k')
    pl.scatter3(ax, foot_l, color='b')
    pl.scatter3(ax, foot_r, color='r')   
    
    # Plot hypotheses
    df_hypo_frame = df_hypo.loc[frame]
    pl.scatter3(ax, df_hypo_frame.L_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)
    pl.scatter3(ax, df_hypo_frame.R_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)

    
    ax.set_xlim3d(-200, 200)
    ax.set_ylim3d(100, 300)
    ax.set_zlim3d(-100, 100)
    
    plt.title('Frame {}'.format(frame))
    
    plt.pause(0.5)

## View 2D frames

In [48]:
frame = 250

_, ax = plt.subplots()
frames = df.index.values

for frame in frames:
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue

    positions = df.loc[frame]

    points = np.stack(positions)
    points_to_plot = points

    head, foot_l, foot_r = points_to_plot

    pl.scatter2(head, color='k')
    pl.scatter2(foot_l, color='b')
    pl.scatter2(foot_r, color='r')    

    # Plot hypotheses
    df_hypo_frame = df_hypo.loc[frame]
    pl.scatter2(df_hypo_frame.L_FOOT, facecolor='none', edgecolor='k')
    pl.scatter2(df_hypo_frame.R_FOOT, facecolor='none', edgecolor='k')

    plt.axis([-200, 200, -100, 80])

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid()

    plt.pause(0.01)